In [1]:
import os
import csv
import requests

In [2]:
# your AgroDataCube access token - replace this code or add the OS environment variable
token = os.environ["AGRODATACUBE_TOKEN"]

In [ ]:
# token to use for AgroDataCube requests
header_token = {"token":token}

# process all points in the file
with open('points.txt', newline='') as csvfile:
    pointsreader = csv.reader(csvfile, delimiter=';', quotechar='"')
    next(pointsreader) # skip header
    for row in pointsreader:
        id = row[1]
        x = row[2]
        y = row[3]
        
        # read all available fields for the point, for all available years
        fields_url = "http://agrodatacube.wur.nl/api/v1/rest/fields?epsg=28992&geometry=POINT({0} {1})&page_size=1000&page_offset=0".format(x, y)

        fields_data = requests.get(fields_url, headers=header_token)
        fields = fields_data.json()

        for field in fields["features"]:
            # basic field data
            field_id = field["properties"]["fieldid"]
            field_year = field["properties"]["year"]
            field_crop = field["properties"]["crop_name"]
            
            # average height for the field
            ahn_url = "http://agrodatacube.wur.nl/api/v1/rest/fields/{0}/ahn".format(field_id)
            ahn_data = requests.get(ahn_url, headers=header_token)
            ahn = ahn_data.json()
            field_ahn_cell_count = ahn["features"][0]["properties"]["count"]
            field_mean_height = 'NA'
            if field_ahn_cell_count != "0":
                field_mean_height = ahn["features"][0]["properties"]["mean"]
            
            # soiltype
            soiltypes_url = "http://agrodatacube.wur.nl/api/v1/rest/fields/{0}/soiltypes".format(field_id)
            soiltypes_data = requests.get(soiltypes_url, headers=header_token)
            soiltypes = soiltypes_data.json()
            soilcodes = [f["properties"]["soilcode"] for f in soiltypes["features"]]

            # ndvi
            ndvi_url = "http://agrodatacube.wur.nl/api/v1/rest/fields/{0}/ndvi?fromdate={1}&todate={2}".format(field_id, field_year, field_year)
            ndvi_data = requests.get(ndvi_url, headers=header_token)
            ndvi_timeseries = ndvi_data.json()
            ndvi = [f["properties"]["ndvi_value"] for f in ndvi_timeseries["features"]]
    
            # list results, probably want to create an output file from it ...
            print(id, ";", x, ";", y, ";", field_year, ";", field_id, ";", field_mean_height, ";", field_crop, ";", soilcodes, ";", ndvi)


1 ; 220390.66130215 ; 451315.159267097 ; 2009 ; 471696 ; 1365.391304347826 ; Mais, corncob mix ; ['Hn21'] ; []
1 ; 220390.66130215 ; 451315.159267097 ; 2010 ; 947149 ; 1365.391304347826 ; Mais, corncob mix ; ['Hn21'] ; []
1 ; 220390.66130215 ; 451315.159267097 ; 2011 ; 1868616 ; 1365.391304347826 ; Aardappelen, consumptie- op zand-/veengrond ; ['Hn21'] ; []
1 ; 220390.66130215 ; 451315.159267097 ; 2012 ; 2691698 ; 1365.391304347826 ; Mais, corncob mix ; ['Hn21'] ; []
1 ; 220390.66130215 ; 451315.159267097 ; 2013 ; 3443038 ; 1365.391304347826 ; Gerst, zomer- ; ['Hn21'] ; [0.1683, 0.2463, 0.2336, 0.2467, 0.2164]
1 ; 220390.66130215 ; 451315.159267097 ; 2014 ; 4270471 ; 1365.391304347826 ; Bieten, suiker- ; ['Hn21'] ; [0.4095, 0.4193, 0.4851, 0.5353, 0.5959]
1 ; 220390.66130215 ; 451315.159267097 ; 2015 ; 5323355 ; 1365.391304347826 ; Mais, snij- ; ['Hn21'] ; [0.1764, 0.2432, 0.1794, 0.178, 0.2592]
1 ; 220390.66130215 ; 451315.159267097 ; 2016 ; 6048554 ; 1365.391304347826 ; Mais, corncob

8 ; 220482.03150812 ; 450569.869396202 ; 2012 ; 2800436 ; 1476.0625 ; Mais, snij- ; ['Hn21'] ; []
8 ; 220482.03150812 ; 450569.869396202 ; 2013 ; 3340324 ; 1476.0625 ; Mais, snij- ; ['Hn21'] ; [0.3018, 0.3972, 0.2826, 0.3157, 0.2911]
8 ; 220482.03150812 ; 450569.869396202 ; 2014 ; 4025158 ; 1476.0625 ; Mais, snij- ; ['Hn21'] ; [0.3506, 0.3333, 0.3571, 0.3698, 0.452]
8 ; 220482.03150812 ; 450569.869396202 ; 2015 ; 5353753 ; 1476.0625 ; Grasland, tijdelijk ; ['Hn21'] ; [0.4347, 0.4953, 0.4563, 0.4653, 0.5909]
8 ; 220482.03150812 ; 450569.869396202 ; 2016 ; 6118102 ; 1476.0625 ; Grasland, tijdelijk ; ['Hn21'] ; [0.5294, 0.5809, 0.6034, 0.6036, 0.55]
8 ; 220482.03150812 ; 450569.869396202 ; 2017 ; 6591488 ; 1476.0625 ; Grasland, tijdelijk ; ['Hn21'] ; [0.792, 0.908, 0.912, 0.92, 0.924]
10 ; 221574.951007828 ; 451723.670118289 ; 2009 ; 485253 ; 1491.7560975609756 ; Mais, snij- ; ['cHn21', 'Hn21', 'pZg23'] ; []
10 ; 221574.951007828 ; 451723.670118289 ; 2010 ; 1498787 ; 1492.027027027027 ; M

16 ; 220923.37340121 ; 452054.193557327 ; 2009 ; 371270 ; 1418.25 ; Grasland, blijvend ; ['cHn23'] ; []
16 ; 220923.37340121 ; 452054.193557327 ; 2010 ; 1598556 ; 1418.25 ; Grasland, blijvend ; ['cHn23'] ; []
16 ; 220923.37340121 ; 452054.193557327 ; 2011 ; 2359874 ; 1418.25 ; Grasland, blijvend ; ['cHn23'] ; []
16 ; 220923.37340121 ; 452054.193557327 ; 2012 ; 2916714 ; 1418.25 ; Grasland, blijvend ; ['cHn23'] ; []
16 ; 220923.37340121 ; 452054.193557327 ; 2013 ; 3576111 ; 1418.25 ; Grasland, blijvend ; ['cHn23'] ; [0.437, 0.4888, 0.4716, 0.5151, 0.6538]
16 ; 220923.37340121 ; 452054.193557327 ; 2014 ; 4400882 ; 1418.25 ; Grasland, blijvend ; ['cHn23'] ; [0.5478, 0.5257, 0.566, 0.5769, 0.638]
16 ; 220923.37340121 ; 452054.193557327 ; 2015 ; 5355458 ; 1418.25 ; Grasland, blijvend ; ['cHn23'] ; [0.5102, 0.5272, 0.4766, 0.5478, 0.6494]
16 ; 220923.37340121 ; 452054.193557327 ; 2016 ; 6171814 ; 1418.25 ; Grasland, blijvend ; ['cHn23'] ; [0.5692, 0.6216, 0.6363, 0.6557, 0.5781]
16 ; 220923.

23 ; 222907.772074356 ; 452135.540583274 ; 2016 ; 5859374 ; 1664.125 ; Mais, snij- ; ['zEZ21'] ; [0.2727, 0.3015, 0.375, 0.3333, 0.35]
23 ; 222907.772074356 ; 452135.540583274 ; 2017 ; 6859114 ; 1664.125 ; Mais, snij- ; ['zEZ21'] ; [0.56, 0.78, 0.356, 0.348, 0.428]
24 ; 222374.475211015 ; 451496.456204688 ; 2009 ; 602712 ; 1551.0263157894738 ; Grasland, blijvend ; ['cHn21', 'pZg23'] ; []
24 ; 222374.475211015 ; 451496.456204688 ; 2010 ; 1146448 ; 1551.0263157894738 ; Grasland, blijvend ; ['cHn21', 'pZg23'] ; []
24 ; 222374.475211015 ; 451496.456204688 ; 2011 ; 2101309 ; 1551.0263157894738 ; Grasland, blijvend ; ['cHn21', 'pZg23'] ; []
24 ; 222374.475211015 ; 451496.456204688 ; 2012 ; 3084630 ; 1551.0263157894738 ; Grasland, blijvend ; ['cHn21', 'pZg23'] ; []
24 ; 222374.475211015 ; 451496.456204688 ; 2013 ; 3916920 ; 1550.5866666666666 ; Groenten open grond (inclusief groentezaden) ; ['cHn21', 'pZg23'] ; [0.1958, 0.2758, 0.2323, 0.247, 0.2307]
24 ; 222374.475211015 ; 451496.456204688 ;

31 ; 221961.864129678 ; 451649.941337348 ; 2013 ; 3232566 ; 1508.7962962962963 ; Grasland, blijvend ; ['cHn21', 'Hn21', 'pZg23'] ; [0.4776, 0.6226, 0.5666, 0.574, 0.5862]
31 ; 221961.864129678 ; 451649.941337348 ; 2014 ; 4079803 ; 1508.7962962962963 ; Grasland, blijvend ; ['cHn21', 'Hn21', 'pZg23'] ; [0.6869, 0.6379, 0.641, 0.6936, 0.7168]
31 ; 221961.864129678 ; 451649.941337348 ; 2015 ; 5278285 ; 1508.7962962962963 ; Grasland, tijdelijk ; ['cHn21', 'Hn21', 'pZg23'] ; [0.5294, 0.5967, 0.5213, 0.6981, 0.625]
31 ; 221961.864129678 ; 451649.941337348 ; 2016 ; 6020480 ; 1508.7962962962963 ; Grasland, blijvend ; ['cHn21', 'Hn21', 'pZg23'] ; [0.568, 0.685, 0.6716, 0.6806, 0.6097]
31 ; 221961.864129678 ; 451649.941337348 ; 2017 ; 6553553 ; 1508.7962962962963 ; Grasland, blijvend ; ['cHn21', 'Hn21', 'pZg23'] ; [0.872, 0.94, 0.92, 0.876, 0.888]
32 ; 222889.809670378 ; 452017.548525638 ; 2009 ; 602713 ; 1680.6705882352942 ; Mais, snij- ; ['cHn21', 'zEZ21'] ; []
32 ; 222889.809670378 ; 452017.54

...

180 ; 221447.2931813 ; 450242.643225043 ; 2010 ; 1060089 ; 1583.0 ; Gerst, zomer- ; ['Hn21'] ; []
180 ; 221447.2931813 ; 450242.643225043 ; 2011 ; 1705407 ; 1583.0 ; Grasland, tijdelijk ; ['Hn21'] ; []
180 ; 221447.2931813 ; 450242.643225043 ; 2012 ; 2965715 ; 1583.0 ; Grasland, tijdelijk ; ['Hn21'] ; []
180 ; 221447.2931813 ; 450242.643225043 ; 2013 ; 3340364 ; 1583.0 ; Grasland, tijdelijk ; ['Hn21'] ; [0.4964, 0.638, 0.5384, 0.5339, 0.5855]
180 ; 221447.2931813 ; 450242.643225043 ; 2014 ; 4025150 ; 1583.0 ; Mais, snij- ; ['Hn21'] ; [0.6347, 0.614, 0.6271, 0.3239, 0.3125]
180 ; 221447.2931813 ; 450242.643225043 ; 2015 ; 5255602 ; 1583.0 ; Mais, snij- ; ['Hn21'] ; [0.275, 0.2839, 0.3777, 0.2888, 0.3103]
180 ; 221447.2931813 ; 450242.643225043 ; 2016 ; 5531493 ; 1583.96875 ; Gerst, zomer- ; ['Hn21'] ; [0.3814, 0.4318, 0.4468, 0.4698, 0.44]
180 ; 221447.2931813 ; 450242.643225043 ; 2017 ; 6298892 ; 1583.96875 ; Grasland, tijdelijk ; ['Hn21'] ; [0.792, 0.904, 0.928, 0.936, 0.828]
181 ; 22

187 ; 220680.82684035 ; 452025.185639128 ; 2010 ; 1598557 ; 1419.2608695652175 ; Mais, snij- ; ['cHn23', 'Hn21', 'pZg23'] ; []
187 ; 220680.82684035 ; 452025.185639128 ; 2011 ; 1800230 ; 1418.8241758241759 ; Mais, snij- ; ['cHn23', 'Hn21', 'pZg23'] ; []
187 ; 220680.82684035 ; 452025.185639128 ; 2012 ; 2916716 ; 1418.8241758241759 ; Mais, snij- ; ['cHn23', 'Hn21', 'pZg23'] ; []
187 ; 220680.82684035 ; 452025.185639128 ; 2013 ; 3606883 ; 1418.8241758241759 ; Aardappelen, zetmeel ; ['cHn23', 'Hn21', 'pZg23'] ; [0.2258, 0.2542, 0.2439, 0.2173, 0.2413]
187 ; 220680.82684035 ; 452025.185639128 ; 2014 ; 4602982 ; 1400.4577464788733 ; Mais, snij- ; ['cHn23', 'Hn21', 'Hn21', 'pZg23'] ; [0.2666, 0.3417, 0.2592, 0.3125, 0.4095]
187 ; 220680.82684035 ; 452025.185639128 ; 2015 ; 5257257 ; 1400.4577464788733 ; Mais, snij- ; ['cHn23', 'Hn21', 'Hn21', 'pZg23'] ; [0.2394, 0.4432, 0.3258, 0.3255, 0.4647]
187 ; 220680.82684035 ; 452025.185639128 ; 2016 ; 5684577 ; 1400.5874125874127 ; Mais, snij- ; ['cH

195 ; 220410.628148637 ; 451413.314317857 ; 2011 ; 1790582 ; 1364.8181818181818 ; Mais, snij- ; ['Hn21'] ; []
195 ; 220410.628148637 ; 451413.314317857 ; 2012 ; 2703685 ; 1364.8181818181818 ; Mais, snij- ; ['Hn21'] ; []
195 ; 220410.628148637 ; 451413.314317857 ; 2013 ; 3887336 ; 1362.0 ; Mais, snij- ; ['Hn21'] ; [0.24, 0.3239, 0.283, 0.2727, 0.2473]
195 ; 220410.628148637 ; 451413.314317857 ; 2014 ; 4318956 ; 1362.0 ; Mais, snij- ; ['Hn21'] ; [0.3555, 0.3157, 0.3809, 0.4065, 0.4105]
195 ; 220410.628148637 ; 451413.314317857 ; 2015 ; 5256382 ; 1362.0 ; Mais, snij- ; ['Hn21'] ; [0.1714, 0.282, 0.2142, 0.225, 0.2876]
195 ; 220410.628148637 ; 451413.314317857 ; 2016 ; 6009556 ; 1362.0 ; Mais, snij- ; ['Hn21'] ; [0.2888, 0.3076, 0.3493, 0.3243, 0.3186]
195 ; 220410.628148637 ; 451413.314317857 ; 2017 ; 6605084 ; 1362.0 ; Mais, snij- ; ['Hn21'] ; [0.42, 0.48, 0.288, 0.316, 0.276]
196 ; 221874.152199199 ; 452236.245038874 ; 2009 ; 791149 ; 1539.5128205128206 ; Grasland, blijvend ; ['Hn21', '

203 ; 220647.82689482 ; 452155.029083381 ; 2012 ; 3121279 ; 1417.7435897435898 ; Grasland, blijvend ; ['cHn23', 'Hn21', 'pZg23'] ; []
203 ; 220647.82689482 ; 452155.029083381 ; 2013 ; 3559372 ; 1417.7435897435898 ; Grasland, blijvend ; ['cHn23', 'Hn21', 'pZg23'] ; [0.4603, 0.5959, 0.5135, 0.5294, 0.625]
203 ; 220647.82689482 ; 452155.029083381 ; 2014 ; 4664748 ; 1417.7435897435898 ; Grasland, blijvend ; ['cHn23', 'Hn21', 'pZg23'] ; [0.6033, 0.5963, 0.5925, 0.6346, 0.6538]
203 ; 220647.82689482 ; 452155.029083381 ; 2015 ; 5180700 ; 1413.7307692307693 ; Grasland, blijvend ; ['cHn23', 'Hn21', 'pZg23'] ; [0.5098, 0.5702, 0.5, 0.4814, 0.647]
203 ; 220647.82689482 ; 452155.029083381 ; 2016 ; 5901150 ; 1413.7307692307693 ; Grasland, blijvend ; ['cHn23', 'Hn21', 'pZg23'] ; [0.4423, 0.6302, 0.6229, 0.6216, 0.5873]
203 ; 220647.82689482 ; 452155.029083381 ; 2017 ; 6865274 ; 1413.7307692307693 ; Grasland, blijvend ; ['cHn23', 'Hn21', 'pZg23'] ; [0.84, 0.932, 0.84, 0.872, 0.78]
204 ; 222808.215715

210 ; 221176.577487563 ; 450678.689148285 ; 2010 ; 1060094 ; 1518.7 ; Grasland, tijdelijk ; ['Hn21'] ; []
210 ; 221176.577487563 ; 450678.689148285 ; 2011 ; 1705408 ; 1518.7 ; Grasland, tijdelijk ; ['Hn21'] ; []
210 ; 221176.577487563 ; 450678.689148285 ; 2012 ; 2800457 ; 1518.7 ; Grasland, tijdelijk ; ['Hn21'] ; []
210 ; 221176.577487563 ; 450678.689148285 ; 2013 ; 3820628 ; 1518.7 ; Mais, snij- ; ['Hn21'] ; [0.4893, 0.3103, 0.3095, 0.315, 0.315]
210 ; 221176.577487563 ; 450678.689148285 ; 2014 ; 4025156 ; 1520.1290322580646 ; Mais, snij- ; ['Hn21'] ; [0.3164, 0.3, 0.317, 0.3055, 0.323]
210 ; 221176.577487563 ; 450678.689148285 ; 2015 ; 5378991 ; 1520.1290322580646 ; Grasland, tijdelijk ; ['Hn21'] ; [0.282, 0.3636, 0.3023, 0.4626, 0.322]
210 ; 221176.577487563 ; 450678.689148285 ; 2016 ; 6118071 ; 1520.1290322580646 ; Grasland, tijdelijk ; ['Hn21'] ; [0.5322, 0.5964, 0.6166, 0.5887, 0.5254]
210 ; 221176.577487563 ; 450678.689148285 ; 2017 ; 6406396 ; 1521.1875 ; Grasland, tijdelijk ; 